In [1]:
import sys
sys.path.insert(1, '..')

import numpy as np
from pandas import isnull
from matplotlib import cm
from matplotlib.colors import to_hex
import seaborn as sns

from drilldown import Renderer, Page, Header, Table, Cell, PageLinkCell

In [2]:
renderer = Renderer('titanic.xlsx')

data = sns.load_dataset('titanic')
data.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [3]:
# Make new cell class with complex behaviour: 
# e.g. one which counts its color and formats its value.
class ColoredCell(Cell):
    def __init__(self, value, max_value=1.0, cmap=cm.viridis, fmt='%.2f'):
        self._value = value
        self._max_value = max_value
        self._cmap = cmap
        self._fmt = fmt
        
    def _get_string(self):
        if isnull(self._value):
            return ''
        return self._fmt % self._value
        
    def _get_color(self):
        if isnull(self._value):
            return
        color_coord = max(0, min(1-1e-9, self._value / self._max_value))
        return to_hex(self._cmap(color_coord))

In [4]:
# Preprocess data: assign age bins.
bins = np.array([0, 5, 10, 15, 20, 25, 30, 
                 35, 40, 45, 50, 55, 60, 65, 70])
bin_keys = ['?'] + [str(b) for b in bins[1:]] + ['>70']

def quantize_age(age):
    if isnull(age):
        return bin_keys[0]
    mask = age <= bins
    if any(mask):
        return bin_keys[mask.nonzero()[0].min()]
    return bin_keys[-1]

data['age_str'] = data.age.apply(quantize_age)

In [5]:
# Assemble the frame with ratios of survivors.
frame = data.pivot_table(index=['pclass', 'sex'], 
                         columns=['age_str'], 
                         values='survived', 
                         aggfunc=np.mean)
# Sort columns.
frame = frame[bin_keys]
# Make colored cells.
frame = frame.applymap(lambda x: ColoredCell(100*x, 
                                             max_value=100, 
                                             fmt='%i%%'))

# Make links out of the first level of indices.
frame.reset_index(inplace=True)
frame['pclass'] = [PageLinkCell(str(x), page_name=f"pclass_{x}") 
                   for x in frame.pclass.values]
frame.set_index(['pclass', 'sex'], inplace=True)

frame

age_str           ?     5    10    15    20   25    30    35    40    45  \
pclass sex                                                                 
1      female  100%    0%        100%  100%  92%  100%  100%  100%  100%   
       male     23%  100%        100%   25%  42%   50%   66%   43%   37%   
2      female  100%  100%  100%  100%  100%  92%   86%  100%   85%   83%   
       male     22%  100%  100%         10%   0%    0%   17%    0%   20%   
3      female   59%   73%    0%   62%   47%  50%   50%   50%   33%    0%   
       male      9%   38%   33%   16%   12%   9%   20%   18%    5%   12%   

age_str          50    55    60    65  70  >70  
pclass sex                                      
1      female   85%  100%  100%  100%           
       male     37%   28%   28%    0%  0%  33%  
2      female  100%  100%    0%                 
       male      0%    0%    0%  100%  0%       
3      female    0%              100%           
       male      0%    0%    0%    0%       0%

In [6]:
# Make top page
top_page = Page(
    'index',
    Header('Titanic dataset exploration', 
           'Percent of suvivors by categories.'),
    Table(frame, 
          column_widths=[10, 20] + [8] * len(frame.columns),
          group_level=0))
renderer.add_page(top_page)

In [7]:
# Make other pages
for pclass in frame.index.unique(level=0):
    pclass_frame = (data.query(f'pclass=="{pclass}"')
                    .set_index(['embark_town', 'who', 'alive'])
                    .sort_index())
    page = Page(f'pclass_{pclass}',
                Header(f'Passengers with class {pclass} tickets', ''),
                Table(pclass_frame,
                      group_level=1,
                      column_widths=([15, 7, 7] 
                                     + [12] * len(pclass_frame.columns)),
                      hidden_columns=[4, 11]),
                parent=top_page)
    renderer.add_page(page)

In [8]:
renderer.render_pages()

C:\Users\Krasnoshekov_KI\AppData\Local\Continuum\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1444: UserWarning: Can't merge single cell
  warn("Can't merge single cell")
